In [1]:
%store -r word_df
%store -r num_df
%store -r single_df
%store -r word_vector

import pandas as pd
import numpy as np

y = word_df['target']
X = num_df.join(single_df)
X = X.drop(columns=['target'])
print(X.head())
print(y.head())

   search_word_Siemens  search_word_cats  search_word_christmas  \
0                    1                 0                      0   
1                    1                 0                      0   
2                    1                 0                      0   
3                    1                 0                      0   
4                    1                 0                      0   

   search_word_donald trump  search_word_dude perfect  search_word_gaming  \
0                         0                         0                   0   
1                         0                         0                   0   
2                         0                         0                   0   
3                         0                         0                   0   
4                         0                         0                   0   

   search_word_golf  search_word_horror  search_word_iphone  \
0                 0                   0                   0   
1       

In [2]:
single_df.columns


Index(['words', 'target'], dtype='object')

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22)



In [5]:
from sklearn.preprocessing import FunctionTransformer


# Obtain the text data: get_text_data
get_text_data = FunctionTransformer(lambda x: x['words'], validate=False)

# Obtain the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(lambda x: x[['search_word_Siemens', 'search_word_cats', 'search_word_christmas',
       'search_word_donald trump', 'search_word_dude perfect',
       'search_word_gaming', 'search_word_golf', 'search_word_horror',
       'search_word_iphone', 'search_word_laugh', 'search_word_mac',
       'search_word_mars', 'search_word_minecraft', 'search_word_movies',
       'search_word_music', 'search_word_python', 'search_word_speedrun',
       'search_word_sports', 'search_word_vacation',
       'search_word_video blogging', 'contentDetails.definition_hd',
       'contentDetails.definition_sd', 'commentCount', 'dislikeCount',
       'likeCount', 'view_bucket', 'age_int', 'title_char_count',
       'title_word_count', 'title_punctuation_count',
       'title_upper_case_word_count', 'title_stopwords_count',
       'desc_char_count', 'desc_word_count', 'desc_word_density',
       'desc_punctuation_count', 'desc_title_word_count',
       'desc_upper_case_word_count', 'desc_stopwords_count', 'tags_char_count',
       'tags_word_count', 'tags_word_density', 'tags_punctuation_count',
       'tags_upper_case_word_count', 'tags_stopwords_count']], validate=False)

# Fit and transform the text data: just_text_data
just_text_data = get_text_data.fit_transform(X_train)

# Fit and transform the numeric data: just_numeric_data
just_numeric_data = get_numeric_data.fit_transform(X_train)

# Print head to check results
print('Text Data')
print(just_text_data.head())
print('\nNumeric Data')
print(just_numeric_data.head())

Text Data
43     Siemens Siemens EQ6 S 300 - Ny elegant espress...
448    dude perfect Baseball Trick Shots | Dude Perfe...
417    dude perfect Old School Trick Shots | Dude Per...
646    mac All Videos From Mac Miller's Secret Instag...
677    christmas Farmhouse Christmas Decor on a Budge...
Name: words, dtype: object

Numeric Data
     search_word_Siemens  search_word_cats  search_word_christmas  \
43                     1                 0                      0   
448                    0                 0                      0   
417                    0                 0                      0   
646                    0                 0                      0   
677                    0                 0                      1   

     search_word_donald trump  search_word_dude perfect  search_word_gaming  \
43                          0                         0                   0   
448                         0                         1                   0   
417         

In [6]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Imputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

process_and_join_features = FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )

# Instantiate nested pipeline: pl
pl = Pipeline([
        ('union', process_and_join_features),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])


# Fit pl to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on all data: ", accuracy)



C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)



Accuracy on all data:  0.45161290322580644


In [7]:
# Randp,
from sklearn.ensemble import RandomForestClassifier

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('clf', RandomForestClassifier())
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy ont dataset: ", accuracy)

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)



Accuracy ont dataset:  0.4959677419354839


In [8]:
from sklearn.linear_model import SGDClassifier
    
pl = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('numeric_features', Pipeline([
                ('selector', get_numeric_data),
                ('imputer', Imputer())
            ])),
            ('text_features', Pipeline([
                ('selector', get_text_data),
                ('tfidf', TfidfVectorizer())
            ]))
            ]
    )),
      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.28225806451612906


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


In [9]:
from sklearn.neighbors import KNeighborsClassifier 

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('knn', KNeighborsClassifier(n_neighbors=11) )
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.5645161290322581


In [10]:
from xgboost import XGBClassifier

pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', CountVectorizer())
                ]))
             ]
        )),
        ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)) 
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)

                     

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.75


In [11]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_GBC = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('tfidf', TfidfVectorizer())
                ]))
             ]
        )),
        ('gbc', GradientBoostingClassifier()) 
    ])

# Fit to the training data
pipe_GBC.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pipe_GBC.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.8508064516129032


In [12]:
from xgboost import XGBClassifier

pipe_XGB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('tfidf', TfidfVectorizer())
                ]))
             ]
        )),
        ('xgb', XGBClassifier(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)) 
    ])

# Fit to the training data
pipe_XGB.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pipe_XGB.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.7620967741935484


In [13]:
# Pipeline with feature importance ability 
from sklearn import feature_selection

pipe_XGB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('tfidf', TfidfVectorizer())
                ]))
             ]
        )),
        ('xgb', XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
 colsample_bytree=0.8, objective= 'multi:softmax', nthread=4, scale_pos_weight=1,seed=27))
    ])

pipe_XGB.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pipe_XGB.score(X_test, y_test)
print("\nAccuracy on dataset: ", accuracy)

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)



Accuracy on dataset:  0.8346774193548387


In [ ]:
# from sklearn.preprocessing import label_binarize
# word_vector_df = pd.DataFrame(word_vector)
# numeric_df = word_vector_df.join(num_df)
# print(numeric_df.columns)

# #temp_y = label_binarize(y, classes=[0, 1, 2, 3, 4, 5])

# X_train, X_test, y_train, y_test = train_test_split(numeric_df, y, random_state=22)
# # print(X_train.columns)
# # print(X_test.columns)
# # print(y_train)
# # print(y_test)

In [18]:
random_XGB = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2)))
                ]))
             ]
        )),
        ('xgb', XGBClassifier(learning_rate =0.1, n_estimators=1000, gamma=0, subsample=0.8,
 colsample_bytree=0.8, objective= 'multi:softmax', nthread=4, scale_pos_weight=1,seed=27))
        ])

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
from sklearn.model_selection import GridSearchCV

param_test1 = {
 'xgb__max_depth':range(3,10,2),
 'xgb__min_child_weight':range(1,6,2)
}

In [20]:
#pass in full pipeline into GridSearchCV( )
gsearch1 = GridSearchCV(random_XGB, param_test1, scoring='accuracy',n_jobs=4,iid=False, cv=5)

#Check objective function for XGBClassifier documentation.  
gsearch1.fit(X_train,y_train)


C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\z002xczx\AppData\Local\Continuu

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x000000000AB19AE8>, inv_kw_args=None,
          inverse_func=None...
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8))]),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'xgb__max_depth': range(3, 10, 2), 'xgb__min_child_weight': range(1, 6, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [21]:
print("Tuned Grid Search Parameters: {}".format(gsearch1.best_params_)) 
print("Best score is {}".format(gsearch1.accuracy_))

Tuned Grid Search Parameters: {'xgb__max_depth': 9, 'xgb__min_child_weight': 1}


AttributeError: 'GridSearchCV' object has no attribute 'accuracy_'

In [ ]:
predictors=list(X_train)
feat_imp = pd.Series(pipe_XGB.steps[0][1].feature_importances_, predictors).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Importance of Features')
plt.ylabel('Feature Importance Score')
print('Accuracy of the GBM on test set: {:.3f}'.format(pipe_XGB.score(X_test, y_test)))
pred=pipe_XGB.predict(X_test)

In [ ]:
# Hyperparameter tuning  on Gradient Boosting Classifier - Set grid search params
parameters = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }
# Construct grid searches
jobs = -1

In [ ]:
# Try Random Forest initially without parameter tuning 
from sklearn.ensemble import RandomForestClassifier

text_rf = Pipeline([
                     ('tfidf', TfidfVectorizer()),
                     ('rf', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)) 
                    ])
text_rf = text_rf.fit(X_train, y_train)

predicted = text_rf.predict(X_test)
print('Accuracy of initial Random Forest: ' + str(np.mean(predicted == y_test)))

In [ ]:
# Again, try random search CV to search optimal hyperparameters
from sklearn.model_selection import RandomizedSearchCV
#Create a random grid for possible parameters to attempt and then use random search

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)



In [ ]:
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, scoring = 'neg_mean_squared_error', n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
#Print and evaluate best parameters from the search
print(rf_random.best_params_)
best_random = rf_random.best_estimator_

In [ ]:
#Evaluate performance of the base model w/ searched hyperparameters
random_model = RandomForestClassifier(n_estimators = 400, min_samples_split=2, min_samples_leaf=1, 
                                   max_features = 'sqrt', max_depth = None, bootstrap = False, random_state = 42)

random_model.fit(X_train, y_train)

In [ ]:
#Evaluate performance of random forest with random searched hyperparameters
from sklearn.metrics import mean_squared_error

print('Random Forest R squared: %.4f' % random_model.score(X_test, y_test))
y_pred = random_model.predict(X_test)
forest_mse = mean_squared_error(y_pred, y_test)
forest_rmse = np.sqrt(forest_mse)
print('Random Forest RMSE: %.4f' % forest_rmse)
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Try N-grams as a feature with MNB

logregpipe = Pipeline([
                      ('vectorizer', TfidfVectorizer(min_df=1, stop_words='english', ngram_range=(1, 2))),
                     ('logreg', LogisticRegression(C=1))
                    ])
logreg = logregpipe.fit(X_train, y_train)
test_predicted = logreg.predict(X_test)
train_predicted = logreg.predict(X_train)

print('Accuracy of Log Reg train set: ' + str(np.mean(train_predicted == y_train)))
print('Accuracy of Log Reg test set: ' + str(np.mean(test_predicted == y_test)))
